In [ ]:
import pandas as pd
import numpy as np
import datatable as dt
import gc
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_color_codes("pastel") # 颜色设定
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
sns.set_color_codes("muted")

In [ ]:
def LoadTables(filename, sepstr, usecols):
    dt_df = dt.fread(filename, sep=sepstr, header=True, columns=usecols, fill=True)
    df  = dt_df.to_pandas()
    del dt_df
    gc.collect()
    return(df)

# Filter
def FilterDF(VDF_DF, region, filter_Frag):
    ## region filter
    print("Befor filter : %d"%len(VDF_DF))
    Pchr = VDF_DF.chrom == region[0]
    Pregion = (VDF_DF.start >= region[1]) & (VDF_DF.end <= region[2])
    VDF_filter = VDF_DF.loc[ Pchr & Pregion , :]
    print("After Region Filter: %d"%len(VDF_filter) )
    ## fragment filter
    Fragmentcount = VDF_filter.groupby(by="read_name", as_index=True)["chrom"].count()
    VDF_filter = VDF_filter.set_index("read_name")
    VDF_filter["Fragnum"] = 0
    VDF_filter.loc[:, "Fragnum"] = Fragmentcount.loc[VDF_filter.index]
    VDF_filter = VDF_filter.loc[VDF_filter.Fragnum>=filter_Frag, :] 
    print("After Fragment number Filter: %d"%len(VDF_filter) )
    return (VDF_filter)

# Bins 
def BinsDF(df, binsize=5000):
    df = df.reset_index()
    df["pos"] = (df.start.values + df.end.values)/2
    df["pos"] = df["pos"].astype("int")
    df["bin"] =  ( df["pos"].values/binsize ).astype("int")
    #df = df.drop(["start", "end", "Fragnum"], axis=1)
    return (df)

# Loading
def Loading(filename, region, filter_Frag, binsize = 5000):
    print("Loading %s"%filename)
    usecols = {"read_name","read_start","read_end","strand","chrom","start","end","MapQual","LRvdF_pfix"}
    VDF_DF = LoadTables(filename, ",",  usecols)
    VDF_DF.columns = ["read_name","read_start","read_end","strand","chrom","start","end","MapQual","LRvdF_pfix"]
    VDF_filter = FilterDF(VDF_DF, region, filter_Frag)
    del(VDF_DF)
    gc.collect()
    # Bin calculate
    bin_df = BinsDF(VDF_filter, binsize)
    del(VDF_filter)
    return(bin_df)

# Loading bed
def Loadingbed(filename, region, binsize, colnames = ["chrom", "start", "end", "value"], usecols = [0,1,2,6] ):
    print("Loading %s"%filename)
    sepstr = "\t"
    bedDF = pd.read_table(filename, sep=sepstr, header=None, usecols=usecols)
    bedDF.columns = colnames
    Pchr = bedDF.chrom == region[0]
    Pregion = (bedDF.start >= region[1]) & (bedDF.end <= region[2])
    bedDF = bedDF.loc[Pchr&Pregion,:]
    bedDF = BinsDF(bedDF, binsize)
    return(bedDF)


In [ ]:
# Loading datas
# Loading read fragment annotation and filter
# Filter 
filter_Frag =  2
binsize = 5000
## region
region = ["chr2", 121340000,121810000]
filename = "chr2_reads_Align_Fragment_RvdF.csv"
binVDF = Loading(filename, region, filter_Frag, binsize)

In [ ]:
# loading CTCF bed
ctcffile="ENCFF796WRU_GM12878_CTCF_narrow_peaks.bed"
bedDF = Loadingbed(ctcffile, region, binsize, ["chrom", "start", "end", "value"] )
bedDF = bedDF.set_index("bin")
bedDF.head()

In [ ]:
# ctcf bin match
ctcfbins = set( bedDF.index.values )
binVDF["binfind"] = 0
binVDF.loc[ binVDF.bin.isin( ctcfbins ),"binfind"] = 1
binVDF.head()

In [ ]:
# Bin Matrix
binsize=5000
readnames = list( set( binVDF.read_name.to_list() ) )
Ts, Te = round(region[1]/binsize), round( region[2]/binsize + 1 )
BinIds = [ idx for idx in range( Ts, Te ) ]

FragMatrix = pd.DataFrame( np.zeros( [len(BinIds), len(readnames)], dtype=int ), 
                          index = BinIds,
                          columns = readnames)
binVDF_group = binVDF.groupby("read_name")
for read_name, df in binVDF_group:
    P = (df.bin>= Ts) & (df.bin<Te)
    FragMatrix.loc[ df.loc[P,"bin"].values, read_name ] = 1

In [ ]:
# Cluster region fragments >= 3
FragSelect="mdLRMFs"
if FragSelect == "All":
    c_frag_num = 2
    cluster_P = FragMatrix.sum() >= c_frag_num
elif FragSelect == "lwLRMFs":
    c_frag_num = 2
    cluster_P = FragMatrix.sum() <= c_frag_num
elif FragSelect == "mdLRMFs":
    cfnl, cfnh = 3, 50
    cluster_P = ( FragMatrix.sum() <= cfnh ) & ( FragMatrix.sum() >= cfnl )
elif FragSelect == "hgLRMFs":
    cfnl = 7
    cluster_P =  FragMatrix.sum() >= cfnl 
    
cluster_reads = cluster_P.loc[cluster_P==True].index.to_list()
FragMatrix_filter = FragMatrix.loc[:, cluster_reads]
print( "%d %s reads in %d all reads in cluster region." %(len(cluster_reads), FragSelect, len(readnames)) )

In [ ]:
FragMatrix_filter.head(10)

In [ ]:
## hierarchy cluster
import seaborn as sns
import scipy
import scipy.cluster.hierarchy as sch

# Cluster
clusterDF = sns.clustermap(data=FragMatrix_filter.T, metric="euclidean", method="ward", figsize=(10,6), col_cluster=False)
# export
## cluster branch tree order
dist_thred = 30
Z = clusterDF.dendrogram_row.calculated_linkage
T = sch.fcluster(Z, t=dist_thred, criterion='distance')
print("The number of clusters: %d"%max(T) )

In [ ]:
# export
## cluster branch tree order
dist_thred = 30
Z = clusterDF.dendrogram_row.calculated_linkage
T = sch.fcluster(Z, t=dist_thred, criterion='distance')
print("The number of clusters: %d"%max(T) )

In [ ]:
## cluster leaves order index
reorder_idx = clusterDF.dendrogram_row.reordered_ind
read_names_order = FragMatrix_filter.iloc[:, reorder_idx].columns.to_list()
## tree branch and leaves diction
Tdic = {}
#T = list(T)[::-1]
for i in range( len(T ) ):
    Tdic[ FragMatrix_filter.columns[i] ] = T[i]
    
# hierarchy cluster_reads 
order_DF = binVDF.set_index("read_name", drop=False).loc[read_names_order]
order_DF["Cluster"] = order_DF["read_name"].apply(lambda x: Tdic[x]) 
readorder = dict()
readidx = 0
for rn in read_names_order:
    readorder[rn] = readidx
    readidx += 1 
order_DF["read_order"] =  order_DF["read_name"].apply(lambda x: readorder[x] )
order_DF = order_DF.reset_index(drop=True)
# start end
order_DF.loc[:, "start"] = (order_DF.bin.values*binsize - 0.5*binsize).astype(int)
order_DF.loc[:, "end"] = (order_DF.bin.values*binsize + 0.5*binsize).astype(int)

In [ ]:
import numpy as np
from numpy import unique
#from sklearn.mixture import GaussianMixture
#X = np.array( FragMatrix_filter.T )
#n_clusters_ = 3
#model = GaussianMixture(n_components=n_clusters_, random_state=10)
#model.fit(X)
#T  = model.predict(X) # labels

from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
X_2d = tsne.fit_transform( FragMatrix_filter.T ) 
plt.figure(figsize=(6, 5))
colors = ["k","g", "b", "orange", "c", "purple",  "y", "navy"]
target_ids = range(1, T.max() + 1 )

legendlist = []
for i, c, label in zip(target_ids, colors[1:], target_ids):
    plt.scatter(X_2d[T == i, 0], X_2d[T == i, 1], c=c, label=label, alpha=0.5)
    legendlist.append("%d : %d"%(i, sum(T==i) ) )
plt.legend(legendlist)
plt.xlabel("tsne1")
plt.ylabel("tsne2")
plt.show()

In [ ]:
legendlist = []
plt.figure(figsize=(3, 10))
for i, c, label in zip(target_ids, colors[1:], target_ids):
    plt.subplot(3,1, i)
    plt.scatter(X_2d[T == i, 0], X_2d[T == i, 1], c=c, label=label, alpha=0.5)
    legendlist.append("%d : %d"%(i, sum(T==i) ) )
    plt.xlim([-100,100])
    plt.ylim([-100,100])
    plt.legend(legendlist[-1])
plt.xlabel("tsne1")
plt.ylabel("tsne2")
plt.show()

In [ ]:
## Reads Cluster Figure
# 统计Cluster reads的数量
def ClusterReads(clusterDF):
    groupDF = clusterDF.groupby("Cluster")
    
    cluster_list, readcount_list = [], []
    sumreads = 0
    readcount = 0
    for cluster, gdF in groupDF:
        cluster_list.append(cluster)
        readcount = len( list(set(gdF["read_name"].to_list()) ) )
        readcount_list.append(readcount)
        sumreads += readcount
    ratio_list = [ float(c)/sumreads for c in readcount_list ]
    cluster_readcount_df = pd.DataFrame({"Cluster":cluster_list,
                                    "readcount":readcount_list,
                                    "ratio":ratio_list})
    return(cluster_readcount_df) 
 
# 统计Cluster Fragment 数量占比 分母是总reads 数 或者是 cluster分群reads数
def FragmentDensity(clusterDF, Total_reads_num):
    # cluster read counts
    cluster_readcount_df  = ClusterReads(clusterDF)
    bincount_df = clusterDF.groupby(["Cluster", "bin"], as_index=False)["read_name"].count()
    bincount_df.columns = ["Cluster", "bin", "count"]
    bincount_df["ratio"] = 0.0
    allbins = list( range( bincount_df["bin"].min(),  bincount_df["bin"].max()+1 ) )
    ## bin ratio = bincount / Total_reads_num
    for i, rcount_row in cluster_readcount_df.iterrows():
        cluster_id,  readcount = rcount_row["Cluster"], rcount_row["readcount"]
        bincount_df.loc[bincount_df.Cluster==cluster_id, "ratio"] = bincount_df.loc[bincount_df.Cluster==cluster_id, "count"].values / Total_reads_num
        # bin without fragments
        bins = bincount_df.loc[bincount_df.Cluster==cluster_id, "bin"].to_list()
        sbins = [ b for b in allbins if b not in bins ]
        sbin_df = pd.DataFrame({"Cluster":len(sbins)*[int(cluster_id)],
                                "bin": sbins,
                               "count":0,
                               "ratio":0.0})
        bincount_df = pd.concat([bincount_df, sbin_df])
    bincount_df = bincount_df.sort_values(by=["Cluster", "bin"], ignore_index=True)
    bincount_df = bincount_df.astype({"Cluster":int, "bin":int})
    # relative ratio  group total reads
    bincount_df.loc[:, "relative_ratio"] = 0.0
    for Cluster, gdf in bincount_df.groupby("Cluster"):
        g_totalreads = Cluster_reads_count.loc[Cluster_reads_count.Cluster==Cluster, "Count"][Cluster]
        P = bincount_df.Cluster == Cluster
        bincount_df.loc[P, "relative_ratio"] = bincount_df.loc[P, "count"].values / g_totalreads
    
    return(bincount_df)

from scipy.interpolate import make_interp_spline
def smoothline(x_array, y_array, smbins=100):
    '''
    smooth lines
    input: x,y array
    output: x, smooth_y array
    bigger smbins makes  
    '''
    xi = np.linspace(x_array.min(),x_array.max(),smbins)
    y_smooth =  make_interp_spline(x_array, y_array)(xi)
    return(xi, y_smooth)

read_order_df = order_DF.groupby(["Cluster","read_name"])["start"].min().reset_index()
read_order_df = read_order_df.sort_values(by=["Cluster","start"], ignore_index=True).reset_index().set_index("read_name")
order_DF.loc[:, "FragY"] = read_order_df.loc[order_DF.read_name.to_list(), "index"].values
colors = ["k","g", "b", "orange", "c", "purple",  "y", "navy"]
subsample = 2 # set subsample reads
plt.figure(figsize=(6,6))
# Cluster Figure 1
ax1 = plt.subplot(5,1,(1,2), rasterized=True ) 
for cluster, gdf in order_DF.groupby(["Cluster", "read_name"]):
    yn = gdf["FragY"].values[0]
    if yn % subsample == 0:
        # grey line
        read_xs = [ gdf["bin"].min()*binsize, gdf["bin"].max()*binsize]
        ys = [yn, yn ]
        #plt.plot(read_xs, ys, c="lightgrey", linewidth=0.2) link lines
        # Fragments
        for indx, rowvalue in gdf.iterrows():
            yn = rowvalue["FragY"]
            sc = colors[cluster[0]]
            #if rowvalue["binfind"] == 1:
            #    sc = "k" # ctct bin fragments black
            start = rowvalue["bin"]*binsize - 0.5*binsize
            end = rowvalue["bin"]*binsize + 0.5*binsize
            xs = [ start, end ]
            ys = [yn, yn ]
            # Fragment
            plt.plot(xs, ys, c=sc, linewidth=0.5)
plt.xticks([])
plt.yticks([])

## Cluster reads density
### Cluster reads number
Cluster_reads_count = order_DF.groupby(["read_name"])["Cluster"].first()
Cluster_reads_count = pd.DataFrame(Cluster_reads_count).reset_index()
Cluster_reads_count.columns = ["Read_name","Cluster"]
summary_count = Cluster_reads_count.Cluster.value_counts()
Cluster_reads_count = pd.DataFrame({"Cluster":summary_count.index,
                                   "Count":summary_count})
Cluster_reads_count.loc[:,"y"] = 0
for n, rowvalue in Cluster_reads_count.iterrows():
    cluster = rowvalue["Cluster"]
    cluster_count = rowvalue["Count"]
    ylist = set( order_DF.loc[order_DF.Cluster == cluster, "read_order"].to_list() )
    yvalue = sum( ylist )/len(ylist)
    Cluster_reads_count.loc[n]["y"] = yvalue
    plt.text(region[2], yvalue,  "Cluster %d\n%d"%(cluster, cluster_count) )

ax2 = plt.subplot(5,1,3, sharex=ax1)
# Calculate Bin cluster read count and bin readcounts
cluster_readcount_df = ClusterReads(order_DF)
#print(cluster_readcount_df)
All_reads_num = Cluster_reads_count.Count.sum() 
BinDensity_df = FragmentDensity(order_DF,All_reads_num)
BinDensity_df.loc[:,"BinPos"] = BinDensity_df.loc[:,"bin"].values * binsize
# Plot Density
groupDF = BinDensity_df.groupby("Cluster")
Clist = []
for cluster, gdf in groupDF:
    cluster = int(cluster)
    x, y = gdf.BinPos.values, gdf.relative_ratio.values
    x, y = smoothline(x, y, 50)
    plt.plot(x, y, c=colors[cluster], linewidth=1)
    Clist.append(cluster)
plt.legend(Clist, loc='upper left')
plt.xlim([region[1], region[2]])
plt.yscale('log')
plt.yticks([0.001, 0.1],["0.001", "0.1"])
plt.ylabel("Fragment\nrelative freq")
plt.xticks([])



# CTCF contact intensity
CTCF_DF = order_DF.loc[order_DF.binfind==1, :]
CTCF_DF.loc[:, "binPos"] =  CTCF_DF.bin.values * binsize
CTCF_DF = CTCF_DF.astype({"Cluster": "str"})
CTCFbins = list( set( CTCF_DF.binPos.to_list() ) )
CTCFbins = sorted(CTCFbins)

### CTCF relative count plot
ctcf_count = CTCF_DF.groupby(["Cluster", "binPos"], as_index=False)["read_name"].count()
ctcf_count.columns = ["Cluster", "binPos", "fcount"]

ax3 = plt.subplot(5,1,4, sharex=ax1)
Clist = []
for cluster, df in ctcf_count.groupby("Cluster"):
    x, y = df.binPos.values, df.fcount.values
    ## calculate relative ctcf percentages = ctcfbins / cluter read numbers
    y = y / summary_count[int(cluster)]
    plt.plot(x,y, linewidth=1.5, color = colors[int(cluster)], alpha=0.5, marker='.', linestyle='-')
    Clist.append(cluster)
plt.legend(Clist, loc='best')
plt.ylabel("CTCF\n relative freq")
plt.yscale('log')
plt.yticks([0.001, 0.1],["0.001", "0.1"])
plt.xticks([])


### CTCF pair contacts
import itertools
Pairs_count = {"Cluster":[],
               "read_name":[],
              "ctcf1":[],
              "ctcf2":[]}
for c1, c2 in itertools.combinations(CTCFbins, 2):
    #print (c1, c2)
    for (cluster, read_name), gdf in CTCF_DF.groupby(["Cluster", "read_name"]):
        binlist = gdf.binPos.to_list()
        if c1 in binlist and c2 in binlist:
            Pairs_count["Cluster"].append(cluster)
            Pairs_count["read_name"].append(read_name)
            Pairs_count["ctcf1"].append(c1)
            Pairs_count["ctcf2"].append(c2)
Pairs_count = pd.DataFrame(Pairs_count)
Pairs_summary = Pairs_count.groupby(["ctcf1", "ctcf2", "Cluster"])["read_name"].count().reset_index()
Pairs_summary.columns = ["ctcf1","ctcf2","Cluster","Count"]

ax4 = plt.subplot(5,1,5, sharex=ax1)
for n, rowvalue in Pairs_summary.iterrows():
    cluster = rowvalue["Cluster"]
    
    i, j  = rowvalue["ctcf1"], rowvalue["ctcf2"]
    middle = (i+j)/2
    widthsize = j-i
    weight =  rowvalue["Count"]
    weight = weight/summary_count[int(cluster)]  # relative height
    
    X = np.linspace(i,j, 50)
    #Y = ( - (X-middle)**2 + ( (widthsize/2)**2 ) )*int(cluster)
    Y = ( - (X-middle)**2 + ( (widthsize/2)**2 ) ) / (widthsize/2)**2 * float(weight)
    plt.plot(X,Y, linewidth=1.5, color = colors[int(cluster)], alpha=0.5)
plt.ylabel("CTCF pair-contact\nrelative freq")


### xticks and save 
# xlim, xticks
plt.xlim([region[1], region[2]])
# xticks
Xtick = list( np.linspace(region[1], region[2], 5 ,endpoint=True) )
Xtick_label = [ "%.3f"%(i/10**6) for i in Xtick ]
plt.xticks( Xtick, Xtick_label)
plt.xlabel("%s Mb"%region[0] )
Figname = "Doublebel"

Exportdir = "%s_%d_%dGM12878"%(region[0], region[1], region[2])
os.system("mkdir -p %s"%Exportdir)
plt.savefig("%s/TAD_clusters_%s:%d-%d.pdf"%(Exportdir,  region[0], region[1], region[2]) )

In [ ]:
# Get Random sample reads
'''
1. Sample reads
2. Get reads with more than ? fragments
'''

def Reorder(df):
    '''
    Reorder read fragment by the first frag
    '''
    re_df = df.groupby(["Cluster","read_name"])["start"].min().reset_index()
    re_df = re_df.sort_values(by=["Cluster","start"], ignore_index=True).reset_index().set_index("read_name")
    df.loc[:, "FragY"] = re_df.loc[df.read_name.to_list(), "index"].values
    return(df)

from sklearn.utils import resample  

Samplenum = 50
Samplerate = 0.50
Fragthred = 6
Pnum = order_DF.Fragnum >= Fragthred
SampleList = [] # Get sample reads
for cluster, gdf in order_DF.loc[Pnum].groupby(["Cluster"], as_index=False):
    Creads = list( set( gdf.read_name.to_list()  ) )  # cluter reads
    Samplenum = int( Samplerate * len(Creads) )
    Sreads = resample(Creads, n_samples=Samplenum,replace=1) # sample reads
    SampleList.append( gdf.loc[gdf.read_name.isin(Sreads) , :] )
Sampledf = pd.concat(SampleList) # Sample df
Sampledf = Reorder(Sampledf)    
 

## Cluster read numbers
Cluster_reads_count = order_DF.groupby(["read_name"])["Cluster"].first()
Cluster_reads_count = pd.DataFrame(Cluster_reads_count).reset_index()
Cluster_reads_count.columns = ["Read_name","Cluster"]
summary_count = Cluster_reads_count.Cluster.value_counts()
Cluster_reads_count = pd.DataFrame({"Cluster":summary_count.index,
                                   "Count":summary_count})    
    
# Cluster Figure 1
#plt.figure(figsize=(8,10) )
FigRow, FigCol, N = 7, 1, 1
plt.subplots(FigRow, FigCol, figsize=(6,12), sharex=True)
Axlist = []
subsample = 1
for cluster, cgdf in Sampledf.groupby("Cluster"):
    Axlist.append( plt.subplot(FigRow, FigCol, N)  )
    N += 1
    for read_name, gdf in cgdf.groupby("read_name"):
        yn = gdf["FragY"].values[0]
        if yn % subsample == 0:
            # grey line
            read_xs = [ gdf["bin"].min()*binsize, gdf["bin"].max()*binsize]
            ys = [yn, yn ]
            #plt.plot(read_xs, ys, c="lightgrey", linewidth=0.2) # fragment link lines
            # Fragments
            for indx, rowvalue in gdf.iterrows():
                yn = rowvalue["FragY"]
                sc = colors[cluster]
                if rowvalue["binfind"] == 1:
                    sc = "k" # ctct bin fragments black
                start = rowvalue["bin"]*binsize - 0.5*binsize
                end = rowvalue["bin"]*binsize + 0.5*binsize
                xs = [ start, end ]
                ys = [yn, yn ]
                # Fragment
                plt.plot(xs, ys, c=sc, linewidth=1)
    plt.xticks([])
    plt.yticks([])
    readnum = summary_count[cluster]
    plt.ylabel( "Cluster %d\nN=%d"%(cluster, readnum) )

    
Axlist.append( plt.subplot(FigRow, FigCol, N)  )
N += 1
# Calculate Bin cluster read count and bin readcounts
cluster_readcount_df = ClusterReads(order_DF)
#print(cluster_readcount_df)
All_reads_num = Cluster_reads_count.Count.sum() 
BinDensity_df = FragmentDensity(order_DF,All_reads_num)
BinDensity_df.loc[:,"BinPos"] = BinDensity_df.loc[:,"bin"].values * binsize
# Plot Density
groupDF = BinDensity_df.groupby("Cluster")
Clist = []
for cluster, gdf in groupDF:
    cluster = int(cluster)
    x, y = gdf.BinPos.values, gdf.relative_ratio.values
    x, y = smoothline(x, y, 100)
    plt.plot(x, y, c=colors[cluster], linewidth=1)
    Clist.append(cluster)
plt.legend(Clist, loc='upper left')
#plt.xlim([region[1], region[2]])
#plt.yscale('log')
#plt.yticks([0.001, 0.1],["0.001", "0.1"])
plt.ylabel("Fragment\nrelative freq")
plt.xticks([])



# CTCF contact intensity
CTCF_DF = order_DF.loc[order_DF.binfind==1, :]
CTCF_DF.loc[:, "binPos"] =  CTCF_DF.bin.values * binsize
CTCF_DF = CTCF_DF.astype({"Cluster": "str"})
CTCFbins = list( set( CTCF_DF.binPos.to_list() ) )
CTCFbins = sorted(CTCFbins)
### CTCF relative count plot
ctcf_count = CTCF_DF.groupby(["Cluster", "binPos"], as_index=False)["read_name"].count()
ctcf_count.columns = ["Cluster", "binPos", "fcount"]
### CTCF pair contacts
import itertools
Pairs_count = {"Cluster":[],
               "read_name":[],
              "ctcf1":[],
              "ctcf2":[]}
for c1, c2 in itertools.combinations(CTCFbins, 2):
    #print (c1, c2)
    for (cluster, read_name), gdf in CTCF_DF.groupby(["Cluster", "read_name"]):
        binlist = gdf.binPos.to_list()
        if c1 in binlist and c2 in binlist:
            Pairs_count["Cluster"].append(cluster)
            Pairs_count["read_name"].append(read_name)
            Pairs_count["ctcf1"].append(c1)
            Pairs_count["ctcf2"].append(c2)
Pairs_count = pd.DataFrame(Pairs_count)
Pairs_summary = Pairs_count.groupby(["ctcf1", "ctcf2", "Cluster"])["read_name"].count().reset_index()
Pairs_summary.columns = ["ctcf1","ctcf2","Cluster","Count"]
## pairs filter 
'''
ctcf pair count >= 3
ctcf pair region >= 25kb
'''
Pairs_summary = Pairs_summary.loc[Pairs_summary.Count >= 3, :]
Psize = abs( Pairs_summary.ctcf1.values - Pairs_summary.ctcf2.values ) >= 25000
Pairs_summary = Pairs_summary.loc[Psize, :]

## CTCF freq figure
Axlist.append( plt.subplot(FigRow, FigCol, N)  )
N += 1
Clist = []
for cluster, df in ctcf_count.groupby("Cluster"):
    x, y = df.binPos.values, df.fcount.values
    ## calculate relative ctcf percentages = ctcfbins / cluter read numbers
    y = y / summary_count[int(cluster)]
    plt.plot(x,y, linewidth=1.5, color = colors[int(cluster)], alpha=0.5, marker='.', linestyle='-')
    Clist.append(cluster)
plt.legend(Clist, loc='best')
plt.ylabel("CTCF\n relative freq")
#plt.yscale('log')
#plt.yticks([0.001, 0.1],["0.001", "0.1"])
plt.xticks([])
#plt.xlim([region[1], region[2]])


## CTCF pairs contact
Axlist.append( plt.subplot(FigRow, FigCol, N)  )
N += 1
for n, rowvalue in Pairs_summary.iterrows():
    cluster = rowvalue["Cluster"]
    
    i, j  = rowvalue["ctcf1"], rowvalue["ctcf2"]
    middle = (i+j)/2
    widthsize = j-i
    weight =  rowvalue["Count"]
    weight = weight/summary_count[int(cluster)]  # relative height
    
    X = np.linspace(i,j, 50)
    #Y = ( - (X-middle)**2 + ( (widthsize/2)**2 ) )*int(cluster)
    Y = ( - (X-middle)**2 + ( (widthsize/2)**2 ) ) / (widthsize/2)**2 * float(weight)
    plt.plot(X,Y, linewidth=1.5, color = colors[int(cluster)], alpha=0.5)
plt.ylabel("CTCF pair-contact\nrelative freq")
#plt.yscale('log')

## CTCF motif orientations
# loading ctcf motif information
ctcf_motif_file="GM12878_CTCF_IDR_peaks.motif.annotation.bed"
motifDF = Loadingbed(ctcf_motif_file, region, binsize, ["chrom", "start", "end","strand"], [0,1,2,5] )
motifDF = motifDF.sort_values(by="pos", ignore_index=True)

Axlist.append( plt.subplot(FigRow, FigCol, N)  )
N += 1
y = 1
for n, rowvalue in motifDF.iterrows():
    x, strand_val = rowvalue["pos"], rowvalue["strand"]
    x_u = 1
    if strand_val == "-":
        x_u = -1
    plt.quiver(x,y,x_u,0, scale=40) # arrow, + , -
    y = y*(-1) # two lines
plt.yticks([])
plt.ylim([-2, 2])
plt.xticks([])
plt.ylabel("CTCF motif\norientation")

flank = 0.05*10**6
plt.xlim([region[1]-flank, region[2]+flank])
# xticks
Xtick = list( np.linspace(region[1], region[2], 5 ,endpoint=True) )
Xtick_label = [ "%.3f"%(i/10**6) for i in Xtick ]
plt.xticks( Xtick, Xtick_label)
plt.xlabel("%s Mb"%region[0] )
#plt.show()

Exportdir = "%s_%d_%dGM12878"%(region[0], region[1], region[2])
os.system("mkdir -p %s"%Exportdir)
plt.savefig("%s/TAD_clusters_%s:%d-%d_v1.pdf"%(Exportdir,  region[0], region[1], region[2]) )

In [ ]:
### CTCF pair contacts
motifdict = {}
for n, rowdf in motifDF.iterrows():
    binID, strand = int(rowdf["bin"]*binsize), rowdf["strand"]
    motifdict[binID] = strand
## ctcf combination pattern()
ctcfPattern ={"--":"tendom",
              "++":"tendom",
              "+-":"convergen",
              "-+":"divergen"}
import itertools
Pairs_count = {"Cluster":[],
               "read_name":[],
              "ctcf1":[],
              "ctcf2":[],
               "ctcf_combin":[],
              "ctcf_pattern":[]}
for c1, c2 in itertools.combinations(CTCFbins, 2):
    #print (c1, c2)
    for (cluster, read_name), gdf in CTCF_DF.groupby(["Cluster", "read_name"]):
        binlist = gdf.binPos.to_list()
        if c1 in binlist and c2 in binlist:
            Pairs_count["Cluster"].append(cluster)
            Pairs_count["read_name"].append(read_name)
            Pairs_count["ctcf1"].append(c1)
            Pairs_count["ctcf2"].append(c2)
            # orientation
            try:
                ctcf_combin = motifdict[c1] + motifdict[c2]
                ctcf_p = ctcfPattern[ctcf_combin]
            except:
                ctcf_combin = "Unknown"
                ctcf_p = "Unknown"
            Pairs_count["ctcf_combin"].append(ctcf_combin)
            Pairs_count["ctcf_pattern"].append(ctcf_p)
            
Pairs_count = pd.DataFrame(Pairs_count)
## filter 20kb ctcf pairs
Psize = abs( Pairs_count.ctcf1.values - Pairs_count.ctcf2.values ) >= 20000
Pairs_count = Pairs_count.loc[Psize, :]
Pairs_count  = Pairs_count.loc[Pairs_count.ctcf_combin != "Unknown", :]

PatternSummary = {"Cluster":[], 
                  "AllelPattern":[]}
for read_name, df in Pairs_count.groupby("read_name"):
    Cluster = df["Cluster"].iloc[0]
    patterns = df["ctcf_pattern"].to_list()
    if len(patterns) >= 2:
        p1 = "multi:"
    else:
        p1 = "single:"
    set_pattern = list( set(patterns) )
    if len(set_pattern) >= 2:
        p2 = "+".join(set_pattern)
    else:
        p2 = set_pattern[0]
    AllelPattern = p1+p2
    PatternSummary["Cluster"].append(Cluster)
    PatternSummary["AllelPattern"].append(AllelPattern )
PatternSummary = pd.DataFrame(PatternSummary)    
PatternSummary  = PatternSummary.sort_values(["Cluster", "AllelPattern"])

In [ ]:
PatternSummary  = PatternSummary.sort_values(["AllelPattern"])
f, ax = plt.subplots(figsize=(6, 6))
sns.despine(f)
sns.histplot(
    PatternSummary,
    x="Cluster", hue="AllelPattern",
    multiple="fill",
    palette="Paired",
    edgecolor=".3",
    linewidth=.5)
plt.ylabel("Percentage")
plt.savefig("%s/TAD_clusters_%s:%d-%d_CTCF_pattern.pdf"%(Exportdir,  region[0], region[1], region[2]) )

In [ ]:
# Export cluster vdFragments
Generate_Contactpy = "Generate_Contact_juiceMatrix_vdF.py"
Generate_coolsh = "Generate_cool.sh"
export_cols =  ["read_name","read_start","read_end","strand","chrom","start","end","MapQual","LRvdF_pfix"]
# clear all vdFragment files coolfiles in the Exoportdir
os.system("mkdir -p %s"%Exportdir)
#os.system( "rm %s/*._juice_matrix.txt"%(Exportdir))
#os.system( "rm %s/*.cool"%(Exportdir))
#os.system( "rm %s/*.mcool"%(Exportdir))

## Generate juicematrix
for cluster, df in  order_DF.groupby("Cluster"):
    inputfile = Exportdir + "/" +"vdFragment.csv"
    juice_matrix = Exportdir + "/" + "C%d_juice_matrix.txt"%(cluster)
    # export vdFragment.csv 
    clt_VDF_DF = df.loc[:, export_cols]
    clt_VDF_DF.to_csv(inputfile, sep=",", header=True, index=False )
    # python <Generate_Contactpy> <inputfile> <juice_matrix> 
    os.system( "python %s %s %s"%(Generate_Contactpy, inputfile, juice_matrix) )  
    os.system( "bash %s %s %s"%(Generate_coolsh, Exportdir, juice_matrix) ) 

In [ ]:
# Genome distance